In [9]:
"""
# Used only for Dev

import simulator, agent, simulator_interface, behavior
from importlib import reload
reload(simulator)
reload(behavior)
reload(simulator_interface)
reload(agent)

"""

'\n# Used only for Dev\n\nimport simulator, agent, simulator_interface, behavior\nfrom importlib import reload\nreload(simulator)\nreload(behavior)\nreload(simulator_interface)\nreload(agent)\n\n'

In [19]:
# debug
import simulator_interface
from importlib import reload
reload(simulator_interface)

from simulator_interface import open_session, close_session
simulator, epuck1, epuck2 = open_session(nb_agents=2)

In [23]:
epuck1.wait(3)

epuck1.left_spd = 3
epuck1.right_spd = 3

epuck1.wait(3)

epuck1.left_spd = 0
epuck1.right_spd = 0

epuck1.wait(3)

In [26]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm", "Tree", "Cup"])
    left_wheel = 1 - right.get('dist', 0)
    right_wheel = 1 - left.get('dist', 0) 
    return left_wheel, right_wheel

In [27]:
epuck1.attach_behavior(obstacle_avoidance, freq=10)
epuck1.start_behavior(obstacle_avoidance)

AttributeError: 'Behavior' object has no attribute 'execute'

In [9]:
close_session(simulator)